In [8]:
import pandas as pd
from google.cloud import bigquery
import numpy as np

In [9]:
new_output_df = pd.read_csv('research_papers_new_processing.csv', index_col=0)
bq_client = bigquery.Client(project='evident-data-dev')
old_output_df = bq_client.list_rows('evident-data-dev.curated_google_scholar.banking_research').to_dataframe()

qa_df = pd.read_csv('research_papers_qa_dataset.csv', index_col=0)


c:\Users\lucas\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
c:\Users\lucas\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
C:\Users\lucas\AppData\Local\Temp\ipykernel_20688\3503487757.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  qa_df = pd.read_csv('research_papers_qa_dataset.csv', index_col=0)


In [22]:
mask = ~old_output_df['paper_url'].isin(new_output_df['paper_url'])

old_not_new_df = old_output_df[mask]
old_not_new_df

,run_month,company_id,company_name,author_url,author_name,paper_url,paper_title,paper_date,paper_citations
10,2025-02-28,1002,Commerzbank,/citations?hl=en&user=TDAL0_EAAAAJ,Ludvík Tesař,/citations?view_op=view_citation&hl=en&user=TD...,gaussian mixture model for texture-based medic...,2015-01-01,0
25,2025-02-28,1003,Santander,/citations?hl=en&user=mZn3e8wAAAAJ,Jose M. Leiva Murillo,/citations?view_op=view_citation&hl=en&user=mZ...,algorithms for maximum-likelihood bandwidth,2015-01-01,0
39,2025-02-28,1004,JPMorganChase,/citations?hl=en&user=-NOTfHIAAAAJ,Sairamesh Konidala,/citations?view_op=view_citation&hl=en&user=-N...,training ai models on sensitive data-the feder...,2015-01-01,17
40,2025-02-28,1004,JPMorganChase,/citations?hl=en&user=-NOTfHIAAAAJ,Sairamesh Konidala,/citations?view_op=view_citation&hl=en&user=-N...,designing and implementing efficient data pipe...,2015-01-01,4
71,2025-02-28,1004,JPMorganChase,/citations?hl=en&user=1WkDG2UAAAAJ,Colin Targonski,/citations?view_op=view_citation&hl=en&user=1W...,a generalized deep learning approach for local...,2019-01-01,87
...,...,...,...,...,...,...,...,...,...
7002,2025-02-28,1064,CommBank,/citations?hl=en&user=V85wBU4AAAAJ,Naomi (Naime) Ranjbar Kermany,/citations?view_op=view_citation&hl=en&user=V8...,incorporating user rating credibility in recom...,2015-01-01,0
7038,2025-02-28,1064,CommBank,/citations?hl=en&user=XkCYSbQAAAAJ,Anna Leontjeva,/citations?view_op=view_citation&hl=en&user=Xk...,do llm personas dream of bull markets? compari...,2015-01-01,0
7064,2025-02-28,1065,ANZ,/citations?hl=en&user=SiydsskAAAAJ,Shampa Shahriyar,/citations?view_op=view_citation&hl=en&user=Si...,inherently edge preserving depth map coding us...,2017-01-01,0
7113,2025-02-28,1067,Westpac,/citations?hl=en&user=flFXjlUAAAAJ,Luci Ellis,/citations?view_op=view_citation&hl=en&user=fl...,"lumps, bumps and waves",2015-01-01,0


In [11]:
companies_to_check = ("HSBC", "CommBank")

In [12]:
lhs_cols = ["company_name", "paper_url"]
rhs_cols = ["authors", 'url', "title", "publication_date", "is_patent", "tags",
            "excluded_tags", "keyword_tags", "has_publication_info"]

In [21]:
two_banks_with_qa = pd.merge(
    old_not_new_df[old_not_new_df['company_name'].isin(companies_to_check)][lhs_cols],
    qa_df[rhs_cols],
    left_on='paper_url',
    right_on = 'url',
    how='left').drop(columns=['url'])

two_banks_with_qa = two_banks_with_qa.replace("[]", np.nan)

def get_exclusion_reason(row):
    reasons = []

    if row['is_patent'] == True:
        reasons.append("is_patent")

    if row['has_publication_info'] == False:
        reasons.append("no_publication_info")

    if pd.notna(row['excluded_tags']):
        reasons.append({"excluded_tags": row['excluded_tags']})

    if pd.isna(row['tags']):
        reasons.append("no_tags")

    if pd.isna(row['keyword_tags']):
        reasons.append("no_keyword_tags")

    if pd.isna(row['authors']):
        reasons.append("no_authors")

    if pd.isna(row['publication_date']):
        reasons.append("no_publication_date")

    return reasons


two_banks_with_qa['exclusion_reason'] = two_banks_with_qa.apply(get_exclusion_reason, axis=1)

two_banks_with_qa

C:\Users\lucas\AppData\Local\Temp\ipykernel_20688\1236834290.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  two_banks_with_qa = two_banks_with_qa.replace("[]", np.nan)


,company_name,paper_url,authors,title,publication_date,is_patent,tags,excluded_tags,keyword_tags,has_publication_info,exclusion_reason
0,HSBC,/citations?view_op=view_citation&hl=en&user=5o...,Maciej Trzetrzelewski,pos (lat2005) 275 the study of su (3) super ya...,2005/7/25,False,NaN,NaN,NaN,True,"[no_tags, no_keyword_tags]"
1,HSBC,/citations?view_op=view_citation&hl=en&user=6d...,"Anusuya Kirubakaran, M Aramudhan",time sensitive business intelligence-big data ...,NaN,False,['ai_keywords'],NaN,['data driven'],False,"[no_publication_info, no_publication_date]"
2,HSBC,/citations?view_op=view_citation&hl=en&user=6d...,"Anusuya Kirubakaran, M Aramudhan",time sensitive business intelligence-big data ...,NaN,False,['ai_keywords'],NaN,['data driven'],False,"[no_publication_info, no_publication_date]"
3,HSBC,/citations?view_op=view_citation&hl=en&user=6d...,"Anusuya Kirubakaran, M Aramudhan",time sensitive business intelligence-big data ...,NaN,False,['ai_keywords'],NaN,['data driven'],False,"[no_publication_info, no_publication_date]"
4,HSBC,/citations?view_op=view_citation&hl=en&user=SE...,"Jai Geelal, Mohamad Khalil, Oxana Samko, Ronni...",an overview of regulations and ethics of artif...,NaN,False,['ai_keywords'],NaN,"['artificial intelligence', ' ai ', 'fraud det...",False,"[no_publication_info, no_publication_date]"
5,HSBC,/citations?view_op=view_citation&hl=en&user=SE...,"Jai Geelal, Mohamad Khalil, Oxana Samko, Ronni...",an overview of regulations and ethics of artif...,NaN,False,['ai_keywords'],NaN,"['artificial intelligence', ' ai ', 'fraud det...",False,"[no_publication_info, no_publication_date]"
6,HSBC,/citations?view_op=view_citation&hl=en&user=Yh...,"Mekena Metcalf, Huo Chen, Mariam Kiran, Anasta...",towards autonomous quantum network control,NaN,False,['ai_keywords'],NaN,"['autonomous', 'reinforcement learning', ' ai ']",False,"[no_publication_info, no_publication_date]"
7,HSBC,/citations?view_op=view_citation&hl=en&user=_U...,"Jaydip Sen, Abhishek Dutta, Sidra Mehtab",portfolio optimization using deep learning mod...,NaN,False,['ai_keywords'],NaN,"['learning based', 'deep learning']",False,"[no_publication_info, no_publication_date]"
8,HSBC,/citations?view_op=view_citation&hl=en&user=_U...,"Jaydip Sen, Abhishek Dutta, Sidra Mehtab",portfolio optimization using deep learning mod...,NaN,False,['ai_keywords'],NaN,"['learning based', 'deep learning']",False,"[no_publication_info, no_publication_date]"
9,HSBC,/citations?view_op=view_citation&hl=en&user=_U...,Abhishek Dutta,facial features identification and object trac...,NaN,False,['ai_keywords'],NaN,['computer vision'],False,"[no_publication_info, no_publication_date]"
